In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-10-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
1471,2024-10-02,Europa Eurocopa,14:00,Buducnost,Gran Canaria,2.50,1.49,163.5,1.85,1.85,3.5,2.00,1.72,bZpGMS2F,0.400000,0.671141,0.540541,0.540541,0.071141,0.4,0.8,NaN,NaN,183.826,41.475847,0.225626,106.204,18.894668,0.177909,160.046,133.506,227.70,113.96,0.0,0.0,0.0,0.0,0.357984,0.000000,0.106446,0.06,0.012,125.000000,0.000000,0.0,0.000000,-2.49,-0.498,-0.983936,0.000000,0.0,0.000000
1472,2024-10-02,Europa Eurocopa,14:30,JL Bourg,Cluj-Napoca,1.15,5.03,165.5,1.85,1.85,-12.5,2.03,1.70,p4dWEy16,0.869565,0.198807,0.540541,0.540541,0.068372,0.8,0.6,NaN,NaN,144.172,34.070890,0.236321,212.456,154.251152,0.726038,149.164,220.940,200.07,118.32,0.0,0.0,0.0,0.0,0.887888,0.000000,0.125118,-0.52,-0.104,-1.442308,0.000000,0.0,0.000000,3.78,0.756,5.330688,0.000000,0.0,0.000000
1473,2024-10-02,Europa Eurocopa,15:00,Trento,Trefl Sopot,1.29,3.37,164.5,1.84,1.86,-8.5,1.98,1.74,QLn8OlY2,0.775194,0.296736,0.543478,0.537634,0.071930,0.4,0.0,NaN,NaN,218.670,128.827480,0.589141,NaN,NaN,NaN,122.742,0.000,128.70,0.00,0.0,0.0,0.0,0.0,0.631237,0.007644,0.091240,-0.87,-0.174,-1.666667,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1474,2024-10-02,Europa Eurocopa,15:30,Valencia,Hamburg,1.06,7.70,167.5,1.85,1.85,-17.5,1.98,1.74,hUavDFVI,0.943396,0.129870,0.540541,0.540541,0.073266,0.0,0.2,NaN,NaN,NaN,NaN,NaN,297.010,169.485391,0.570639,0.000,316.528,0.00,592.00,0.0,1.0,0.0,0.0,1.071961,0.000000,0.091240,0.00,0.000,inf,0.000000,0.0,0.000000,-2.88,-0.576,-11.631944,0.333749,0.3,-0.033749
1475,2024-10-02,Alemanha Bbl,15:00,Ludwigsburg,Rostock,1.29,3.66,159.5,1.80,1.94,-9.5,2.03,1.73,fwxQIE6s,0.775194,0.273224,0.555556,0.515464,0.048418,0.2,0.6,NaN,NaN,196.410,75.317878,0.383473,174.546,30.197233,0.173004,193.254,207.830,259.74,220.11,0.0,0.0,0.0,0.0,0.677108,0.052938,0.112836,0.52,0.104,2.788462,0.581707,0.6,0.018293,-0.57,-0.114,-23.333333,0.375002,0.3,-0.075002
1476,2024-10-02,Ásia Easl,07:10,San Miguel Beermen,Suwon KT,1.72,1.95,166.5,1.83,1.83,-2.5,1.99,1.70,WhImxiLk,0.581395,0.512821,0.546448,0.546448,0.094216,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.088629,0.000000,0.111144,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1477,2024-10-02,Ásia Easl,09:10,Meralco Bolts,Macau Black Bears,1.47,2.43,165.5,1.93,1.74,-5.5,1.97,1.71,G2TWAEMj,0.680272,0.411523,0.518135,0.574713,0.091795,0.0,0.0,NaN,NaN,332.208,213.843619,0.643704,NaN,NaN,NaN,0.000,0.000,250.25,0.00,0.0,0.0,0.0,0.0,0.348114,0.073215,0.099917,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
1478,2024-10-02,Eslováquia Extraliga,12:30,Nitra,BKM Lucenec,2.43,1.47,161.5,1.83,1.83,3.5,1.97,1.71,Cxa6BMUj,0.411523,0.680272,0.546448,0.546448,0.091795,0.6,1.0,NaN,NaN,469.892,178.578249,0.380041,289.186,277.049467,0.958032,401.052,345.034,361.20,777.40,1.0,1.0,0.0,0.0,0.348114,0.000000,0.099917,-0.59,-0.118,-12.118644,0.000000,0.0,0.000000,-5.00,-1.000,-0.470000,0.000000,0.0,0.000000
1479,2024-10-02,Eslováquia Extraliga,13:00,BC Komarno,Iskra Svit,1.66,2.03,160.5,1.80,1.86,-3.5,1.98,1.70,SpsgZPMc,0.602410,0.492611,0.555556,0.537634,0.095020,0.2,0.6,NaN,NaN,146.852,48.031906,0.327077,140.896,20.007537,0.142002,163.052,189.878,214.72,108.29,0.0,0.0,0.0,0.0,0.141805,0.023184,0.107603,0.84,0.168,3.928571,0.000000,0.0,0.000000,1.00,0.200,5.150000,0.000000,0.0,0.000000
1480,2024-10-02,Eslováquia Extraliga,13:00,Inter Bratislava,Spisski Rytieri

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,09:10,Ásia Easl,Meralco Bolts,Macau Black Bears,165.5,1.93,0.7385,Over
